# Analysis Configuration & Constants



Data sourced from the [2023 US ADP Report](https://www.evolved.energy/2023-us-adp) and exported into CSV format.

In [19]:
# filepath to ECapacity data (CSV)
data_path = 'data/ECapacity.csv'

# Export filepath (JSON)
export_path = 'data/projects.json'

# policy name (as specified in CSV file/ADP report)
policy = 'Central'

# year range to analyze
start_year = 2021
end_year = 2050

## Gigawatts Per Project
A dictionary to convert between project type and GW produced per project
[(source, in MW)](https://docs.google.com/presentation/d/1-2MsOicq1c1aPRusFySC3uiG0ezH5gRE/edit#slide=id.p26).

You can choose to specify as many project types as you want. Leave the ones that you do not wish to analyze with a ``None`` value (these project types will be ignored in the analysis).

In [20]:
# Megawatts produced per project (on average)
project_to_MW = {
    'Customer-sited Solar': None,
    'Transmission-sited Solar': 543.0,
    'Offshore Wind': 1184.0,
    'Onshore Wind': 724.0,
    'Biomass w/CC': None,
    'Gas w/CC': 868.0,  # Assume new energy production will use only CCGT
    'Gas': None,
    'Coal': None,
    'Nuclear': None,
    'Hydro': None,
    'Other': None
}

# Convert values to Gigawatts, remove None values
MW_TO_GW = 1/1000.0
project_to_GW = dict()
for k, v in project_to_MW.items():
    if v is not None:
        project_to_GW[k] = v * MW_TO_GW

project_to_GW

{'Transmission-sited Solar': 0.543,
 'Offshore Wind': 1.184,
 'Onshore Wind': 0.724,
 'Gas w/CC': 0.868}

## Duration Per Project
A dictionary to convert between project type and total duration of project
[(source, in weeks)](https://docs.google.com/presentation/d/1-2MsOicq1c1aPRusFySC3uiG0ezH5gRE/edit#slide=id.p16).

Again, specify as many projects as you wish (the rest will be ignored).

In [21]:
# Duration (in weeks) per project (on average)
project_duration_weeks = {
    'Customer-sited Solar': None,
    'Transmission-sited Solar': 312.0,
    'Offshore Wind': 390.0,
    'Onshore Wind': 312.0,
    'Biomass w/CC': None,
    'Gas w/CC': 364.0,
    'Gas': None,
    'Coal': None,
    'Nuclear': None,
    'Hydro': None,
    'Other': None
}

# Convert values to years, remove None values
WEEK_TO_YEAR = 1/52.0
project_duration = dict()
for k, v in project_duration_weeks.items():
    if v is not None:
        project_duration[k] = v * WEEK_TO_YEAR

if project_duration.keys() != project_to_GW.keys():
    raise Exception("Need to specify both MW/projects and keys")

project_duration

{'Transmission-sited Solar': 6.0,
 'Offshore Wind': 7.5,
 'Onshore Wind': 6.0,
 'Gas w/CC': 7.0}

# Import and Clean Data

In [22]:
import pandas as pd
df = pd.read_csv(data_path)

# simplify column names
df.columns = ['Policy', 'Tech', 'State', 'Year', 'Value', 'Unit']

# remove NaN values
df = df.dropna()

# select policy (ex: Central)
df = df.loc[df['Policy'] == policy]

# relabel indices
df = df.reset_index(drop=True)

# convert Year to int
df['Year'] = df['Year'].astype(int)

In [23]:
# sample and view data
df.sample(10)

,Policy,Tech,State,Year,Value,Unit
3152,Central,Other,maine,2021,1.092,gigawatt
1886,Central,Gas,kansas,2025,3.542,gigawatt
2199,Central,Coal,georgia,2040,5.401,gigawatt
674,Central,Transmission-sited Solar,vermont,2040,0.405,gigawatt
2746,Central,Hydro,hawaii,2045,0.011,gigawatt
875,Central,Offshore Wind,nebraska,2035,0.000,gigawatt
433,Central,Transmission-sited Solar,hawaii,2021,0.284,gigawatt
1395,Central,Biomass w/CC,florida,2050,0.021,gigawatt
331,Central,Customer-sited Solar,washington,2030,1.091,gigawatt
116,Central,Customer-sited Solar,kansas,2040,0.760,gigawatt


# Total Number of Projects Needed by 2050

**Assumption: projects are not retired after they are built.**

Thus, the capacity at 2050 minus the capacity at 2021 represents the total capacity built. Then, by dividing by the capacity per project (depending on type of project), we can figure out how many projects are built.



In [24]:
'''
Group all project capcaity by "tech" (project type) in a given year.
'''
def groupProjectsByTechInYear(df, year):
    # Select the target year
    df_year = df.loc[df['Year'] == year]

    # Sum states together, keep tech separate
    df_year = df_year.groupby(['Tech', 'Year']).sum().reset_index()

    # drop unused columns
    df_year = df_year.drop(['Policy', 'Year', 'State', 'Unit'], axis=1)

    return df_year.set_index('Tech')

In [25]:
'''
Of the specified projects (known project MW value), make sure each is in the
dataframe, otherwise fill with 0.
'''
def ensure_all_techs_present(data):
    for tech in project_to_GW.keys():
        if tech not in data.index:
            data.loc[tech] = 0
    return data

In [26]:
'''
Find the total number of projects needed between start and end years
'''
def num_projects_needed(df, start_interval=start_year, end_interval=end_year):
    # group projects by tech in a given year
    df_start = groupProjectsByTechInYear(df, start_interval)
    df_end = groupProjectsByTechInYear(df, end_interval)

    # ensure all techs are present
    df_start = ensure_all_techs_present(df_start)
    df_end = ensure_all_techs_present(df_end)

    # find difference between values of end and start for each respective tech column
    diff = df_end.sub(df_start, fill_value=0)

    # drop Tech rows in diff not in project_to_GW.keys()
    num_projects = diff.loc[project_to_GW.keys()]

    # divide by project_to_GW to get number of projects
    num_projects = num_projects.div(project_to_GW, axis=0)
    num_projects.columns = ['# Projects Needed']

    return num_projects

In [ ]:
num_projects = num_projects_needed(df, start_year, end_year)
num_projects

# Number of Projects Needed by Year

This is the same approach as calculating the total number of projects needed, but keeping the projects separated by year built. Instead of taking the difference over the entire period ($c_{2050} - c_{2021}$), we take the difference between the start and end of each interval.

In [ ]:
years = [2021, 2025, 2030, 2035, 2040, 2045, 2050]

num_by_year = pd.DataFrame()
for i in range(len(years)-1):
    start_interval = years[i]
    end_interval = years[i+1]

    num_interval = num_projects_needed(df, start_interval, end_interval)
    num_interval.columns = [str(start_interval)]

    num_by_year = pd.concat([num_by_year, num_interval], axis=1)

# add a total column
num_by_year['Total by 2050'] = num_by_year.sum(axis=1)

num_by_year

# Exporting Projects

Given the number of projects needed (as calculated above), generate a ``projects`` dictionary according to the format from ``MILP-Solution.ipynb``. Export as JSON so ``MILP-Solution.ipynb`` can read it to find the optimal schedule.

Format:
```
projects = {
    id#: {"name": project_type, "duration": duration, "dependencies": [id#, id#, ...]},
    ...
}
```

Example:
```
projects = {
    '1': {"name": "Wind Farm", "duration": 1.0, "dependencies": []},
    '2': {"name": "H2", "duration": 1.0, "dependencies": []},
    '3': {"name": "Solar Farm", "duration": 1.0, "dependencies": [2]},
    '4': {"name": "Solar Farm", "duration": 2.0, "dependencies": [1, 3]},
    '5': {"name": "H2 Plant", "duration": 3.0, "dependencies": [1, 2, 6]},
    '6': {"name": "Wind Farm", "duration": 1.0, "dependencies": [1]},
}
```

The "Year ___" projects are virtual item that represents the end of each time interval. This is useful because then we can have certain projects depend on that interval "finishing" (i.e. we can make projects that don't begin until a certain year).

In [ ]:
# make a csv of projects based on # needed
projects = {}
index = 0

# add projects representing the end of the year
year_to_index = dict(zip(years, [index for index, year in enumerate(years)]))
for year in years:
    projects[index] = {"name": f"Year {year}", "duration": year - 2021, "dependencies": []}
    index += 1

# add actual projects, dependending on what year they need to start
for year in years[:-1]:
    for category in num_by_year.index:
        for i in range(int(num_by_year[str(year)][category])):
            projects[index] = {"name": category, "duration": project_duration[category], "dependencies": [year_to_index[year]]}
            # projects[index] = {"name": category, "capacity": #, "duration": project_duration[category], "dependencies": [year_to_index[year]]}
            index += 1

# export dictionary to json
import json
with open(export_path, 'w') as f:
    json.dump(projects, f)